# Tracy Widom test
Implementation according to Patterson 2006, PLoS Genetics.
Variable names are in accordance as much as possible.

In [3]:
import numpy as np
from TracyWidom import TracyWidom
import scipy
import pandas as pd
import matplotlib.pyplot as plt

## Sample dataset
C is a 50x400 genotype  values with values $\in \{0, 1, 2\}$ from the [LEA tutorial](https://rdrr.io/bioc/LEA/man/main_tracyWidom.html).

In [4]:
C = pd.read_csv('Data/genotype.csv').iloc[:,1:].values

## Implementation ala Patterson
### Scaled raw input: Matrix M
Equation (1)-(3) in [Patterson et al, 2006](https://journals.plos.org/plosgenetics/article?id=10.1371/journal.pgen.0020190#pgen-0020190-e003)

$$\mu(j) = \frac{\sum_{i=1}^{m}C(i,j)}{m}$$
$$p(j) = \mu(j)/2$$
$$M(i,j) = \frac{C(i,j)-\mu(j)}{\sqrt{p(j)(1-p(j))}}$$

In [5]:
m, n = C.shape
mu = np.nanmean(C, axis=0)  #(1)
p = mu/2.
scale = np.sqrt(p*(1-p))
#scale = np.nanstd(C, axis=0) ## alternative scaling with sigma, probably used in LEA
M = (C-mu)/scale #(2) & (3) 

### Eigenvalues of X
Seems unneccessary to calculate $X = MM'$ if we go with SVD.

Automatically sorted with linalg.svd, such that $\lambda_1 > \lambda_2 \ldots > \lambda_{m'} > 0$


In [6]:
## through SVD
U, s, V = scipy.linalg.svd(M) 
lambdas = (s**2)[:-1]
# U contains eigenvectors, identical with LEA tutorial :-D


### Moment estimator
Equation (10) in Patterson et al.

$$n' = \frac{(m+1)(\sum_{i}\lambda_i)^2}
{(m-1)\sum_{i}\lambda_i^2 - (\sum_{i}\lambda_i)^2}$$

In [7]:
def nprime(m, lambdas): ## Eq (10)
    import pdb
    t1 = (lambdas.sum())**2
    numer = (m+1) * t1
    denom = (m-1) * (lambdas **2).sum() - t1
    if numer/denom < 0: pdb.set_trace()
    return numer/denom

In [8]:
def twstats(lambdas):
    import pdb
    tw = TracyWidom(beta=1)
    stats = []
    for m in range(len(lambdas)+1, 2, -1):
        m1 = m - 1 ## m' in Patterson
        n1 = nprime(m, lambdas)
        mumn = ((np.sqrt(n1-1) + np.sqrt(m1))**2)/n1 ## Eq (5) ## ERROR in Patterson m', not m!!!
        sigmn = (np.sqrt(n1-1) + np.sqrt(m1))/n1 * (1/np.sqrt(n1-1) + 1/np.sqrt(m1))**(1/3.) # Eq (6)
        l = m1*lambdas[0]/lambdas.sum() ## extend to all lambdas
        x = (l - mumn)/sigmn  # Eq (7)
        stats.append((lambdas[0], l, x, 1-tw.cdf(x)))
        lambdas = lambdas[1:]  ## dropping first lambda, preparing for next round
    df = pd.DataFrame(stats)
    df.columns = 'lambda l twstat p-value'.split()
    return df


In [17]:
result = twstats(lambdas)

In [13]:
## Comparing Eigenvalues/TW stats to smartPCA log
import os
os.chdir('Data')
!smartpca -p pca.param
os.chdir('..')

parameter file: pca.param
### THE INPUT PARAMETERS
##PARAMETER NAME: VALUE
genotypename: genotypes.geno
snpname: genotypes.snp
indivname: genotypes.ind
evecoutname: genotypes_A.pcs.txt
evaloutname: genotypes_A.pve.txt
popfile: pops.txt
altnormstyle: NO
outliersigmathresh: 600.0
lsqproject: YES
## smartpca version: 16000
norm used

lsqproject used
number of samples used: 50 number of snps used: 400
Using 31 threads, and partial sum lookup algorithm.
total number of snps killed in pass: 0  used: 400

## Tracy-Widom statistics: rows: 50  cols: 400
  #N    eigenvalue  difference    twstat      p-value effect. n
   1      5.673028          NA    13.145  1.17602e-15    60.106
   2      4.445142   -1.227886    20.024  7.59313e-28   101.399
   3      2.174567   -2.270576    10.292  2.17171e-11   243.670
   4      1.743260   -0.431307     5.577  5.64185e-06   304.751
   5      1.545026   -0.198233     3.250   0.00104568   345.064
   6      1.325417   -0.219609    -1.266     0.499339   377.018
 

## Comparison of TW stats
Strangely, eigenvalues and TW stats using above code, smartpca and R package LEA are similar but not identical:

In [23]:
# head to head comparison: This implementation vs smartPCA log
result_smartpca = pd.read_csv('Data/smartpca.log', delim_whitespace=True).iloc[:,[1,3,4]]
result_smartpca.columns = [f'SM_{col}' for col in result_smartpca.columns.values]

In [25]:
# compare tw-stats with original eigenvalues from smartPCA
results1 = twstats(np.array(result_smartpca['SM_eigenvalue']))
results1.columns = [f'TW_{col}' for col in results1.columns.values]

In [26]:
pd.concat([result, result_smartpca,results1], axis=1).head(20)

,lambda,l,twstat,p-value,SM_eigenvalue,SM_twstat,SM_p-value,TW_lambda,TW_l,TW_twstat,TW_p-value
0,4963.372596,5.342401,13.096062,4.440892e-16,5.673028,13.145,1.176020e-15,5.673028,5.673028,13.145439,3.330669e-16
1,4017.848721,4.754825,20.024217,0.000000e+00,4.445142,20.024,7.593130e-28,4.445142,4.924573,20.024311,0.000000e+00
2,2001.556972,2.574359,9.716733,4.373502e-11,2.174567,10.292,2.171710e-11,2.174567,2.628597,10.291844,6.795675e-12
3,1625.527437,2.164809,5.274234,1.238042e-05,1.743260,5.577,5.641850e-06,1.743260,2.184580,5.576972,5.894635e-06
4,1423.877420,1.946648,2.310573,6.096839e-03,1.545026,3.250,1.045680e-03,1.545026,1.988507,3.249767,1.045470e-03
5,1235.120345,1.725718,-1.853800,6.833253e-01,1.325417,-1.266,4.993390e-01,1.325417,1.745067,-1.266496,4.993249e-01
6,1219.691623,1.733416,-1.146872,4.615711e-01,1.275327,-1.590,6.022330e-01,1.275327,1.708725,-1.589839,6.022124e-01
7,1150.701429,1.664433,-2.289849,8.015867e-01,1.210664,-2.545,8.577520e-01,1.210664,1.649929,-2.545136,8.577231e-01
8,1109.647317,1.631490,-2.668959,8.809737e-01,1.190185,-2.147,7.655770e-01,1.190185,1.648146,-2.146641,7.655481e-01
9,1066.036803,1.592511,-3.255672,9.567771e-01,1.134034,-3.024,9.332710e-01,1.134034,1.596254,-3.023840,9.332333e-01


In [1]:
#LEA implementation for comparison
l='''N eigenvalues twstats   pvalues      effectn percentage
1   1      2057.0 13.3200 8.000e-09 7.170617e+01   0.104900
2   2      1675.0 20.0100 8.000e-09 1.155594e+02   0.085440
3   3       864.5  9.9680 8.000e-09 2.563951e+02   0.044110
4   4       682.5  4.1770 1.503e-04 3.173119e+02   0.034820
5   5       603.4  1.3000 3.152e-02 3.508808e+02   0.030790
6   6       548.6 -1.0170 4.215e-01 3.730542e+02   0.027990
7   7       522.2 -1.7650 6.565e-01 3.861965e+02   0.026640
8   8       506.0 -1.8630 6.859e-01 3.968453e+02   0.025810
9   9       492.0 -1.8220 6.738e-01 4.076199e+02   0.025100
10 10       464.5 -3.0520 9.363e-01 4.191613e+02   0.023700'''

